In [19]:
# Specialist Finders

%matplotlib inline

import os

import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
import theano


def float32(k):
    return np.cast['float32'](k)

class AdjustVariable(object):
    def __init__(self, name, start=0.03, stop=0.001):
        self.name = name
        self.start, self.stop = start, stop
        self.ls = None

    def __call__(self, nn, train_history):
        if self.ls is None:
            self.ls = np.linspace(self.start, self.stop, nn.max_epochs)

        epoch = train_history[-1]['epoch']
        new_value = float32(self.ls[epoch - 1])
        getattr(nn, self.name).set_value(new_value)




def load(test=False, cols=None):
    #FTRAIN = '/home/ec2-user/W207FinalProject/Project/Data/testAugBig2.csv'    
    FTRAIN = '/home/ec2-user/W207FinalProject/Project/Data/training.csv'
    FTEST = '/home/ec2-user/W207FinalProject/Project/Data/test.csv'

    """Loads data from FTEST if *test* is True, otherwise from FTRAIN.
    Pass a list of *cols* if you're only interested in a subset of the
    target columns.
    """
    fname = FTEST if test else FTRAIN
    #df = read_csv(os.path.expanduser(fname))  # load pandas dataframe
    df = read_csv(fname)  # load pandas dataframe

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    if cols:  # get a subset of columns
        df = df[list(cols) + ['Image']]

    print(df.count())  # prints the number of values for each column
    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['Image'].values) / 255.  # scale pixel values to [0, 1]
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[:-1]].values
        y = y.astype(np.float32)
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        
    else:
        y = None

    return X, y


#X, y = load()
#print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(
#    X.shape, X.min(), X.max()))
#print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(
#    y.shape, y.min(), y.max()))

def load2d(test=False, cols=None):
    X, y = load(test=test,cols=cols)
    X = X.reshape(-1, 1, 96, 96)
    return X, y

############################################################################################
############################################################################################

class SpecialistNet(object):

    def __init__(self,col_num):
        
        self.train_X, self.train_y = load2d(test = False, cols = col_num)
        self.build_net()
        self.train_net()
        
    def build_net(self):
        self.thisNet = NeuralNet(
            layers=[
                ('input', layers.InputLayer),
                ('conv1', layers.Conv2DLayer),
                ('pool1', layers.MaxPool2DLayer),
                ('conv2', layers.Conv2DLayer),
                ('pool2', layers.MaxPool2DLayer),
                ('conv3', layers.Conv2DLayer),
                ('pool3', layers.MaxPool2DLayer),
                ('hidden4', layers.DenseLayer),
                ('hidden5', layers.DenseLayer),
                ('hidden6', layers.DenseLayer),
                ('output', layers.DenseLayer),
                ],

            input_shape=(None, 1, 96, 96),
            conv1_num_filters=16, conv1_filter_size=(4, 4), pool1_pool_size=(2, 2),
            conv2_num_filters=24, conv2_filter_size=(3, 3), pool2_pool_size=(2, 2),
            conv3_num_filters=32, conv3_filter_size=(2, 2), pool3_pool_size=(2, 2),
            hidden4_num_units=500, hidden5_num_units=100, hidden6_num_units=30,
            output_num_units=1, output_nonlinearity=None,

            update_learning_rate=theano.shared(float32(0.1)),
            update_momentum=theano.shared(float32(0.9)),

            regression=True,
            #batch_iterator_train = WonkyBatchIterator(batch_size=128),

            on_epoch_finished=[
            AdjustVariable('update_learning_rate', start=0.1, stop=0.0001),
            AdjustVariable('update_momentum', start=0.9, stop=0.999),],

            max_epochs=250,
            verbose=1,
            )

    def train_net(self):

        self.thisNet.fit(self.train_X, self.train_y)

    def predict(self):
        test_X = load2d(test=True)
        self.predictions = self.thisNet.predict(test_X)

        return self.predictions

        
############################################################################################
############################################################################################
colnames = [u'left_eye_center_x', u'left_eye_center_y', u'right_eye_center_x',
       u'right_eye_center_y', u'left_eye_inner_corner_x',
       u'left_eye_inner_corner_y', u'left_eye_outer_corner_x',
       u'left_eye_outer_corner_y', u'right_eye_inner_corner_x',
       u'right_eye_inner_corner_y', u'right_eye_outer_corner_x',
       u'right_eye_outer_corner_y', u'left_eyebrow_inner_end_x',
       u'left_eyebrow_inner_end_y', u'left_eyebrow_outer_end_x',
       u'left_eyebrow_outer_end_y', u'right_eyebrow_inner_end_x',
       u'right_eyebrow_inner_end_y', u'right_eyebrow_outer_end_x',
       u'right_eyebrow_outer_end_y', u'nose_tip_x', u'nose_tip_y',
       u'mouth_left_corner_x', u'mouth_left_corner_y', u'mouth_right_corner_x',
       u'mouth_right_corner_y', u'mouth_center_top_lip_x',
       u'mouth_center_top_lip_y', u'mouth_center_bottom_lip_x',
       u'mouth_center_bottom_lip_y']

allNets = []
for col in colnames:
    allNets.append(SpecialistNet([col]))

import cPickle as pickle
import datetime as dt
now = str(dt.datetime.now()).replace(' ','_').split('.')[0]
with open('/mnt/Specialized_CNN' + now + '.pickle', 'wb') as f:
    pickle.dump(allNets, f, -1)

#xx,yy = load2d(test=False,cols=[colnames[1]])

left_eye_center_x    7039
Image                7049
dtype: int64
# Neural Network with 1658333 learnable parameters

## Layer information

  #  name     size
---  -------  --------
  0  input    1x96x96
  1  conv1    16x93x93
  2  pool1    16x46x46
  3  conv2    16x44x44
  4  pool2    16x22x22
  5  conv3    32x21x21
  6  pool3    32x10x10
  7  hidden4  500
  8  hidden5  100
  9  hidden6  30
 10  output   1

  epoch    train loss    valid loss    train/val  dur
-------  ------------  ------------  -----------  -----
      1       0.01062       0.00477      2.22631  7.44s
      2       0.00523       0.00470      1.11311  7.43s
      3       0.00516       0.00461      1.11909  7.43s
      4       0.00506       0.00445      1.13525  7.44s
      5       0.00487       0.00421      1.15881  7.44s
      6       0.00464       0.00398      1.16433  7.43s
      7       0.00437       0.00371      1.17835  7.43s
      8       0.00407       0.00347      1.17404  7.44s
      9       0.00375       0.0

In [110]:
Xtest,_ = load2d(test=True)
#bork = SpecialistNet([colnames[-5]])
#Xtest,_ = load2d(test=True)


ImageId    1783
Image      1783
dtype: int64


In [115]:
darf = allNets[-6].thisNet.predict(Xtest)
print darf
#vv.train_y=y
#vv.train_X=X
#vv.train_net()
#vv.build_net()
#vv.train_net()
#darf = vv.predict()

[[-0.31271091]
 [-0.30356765]
 [-0.1882281 ]
 ..., 
 [-0.32356617]
 [-0.35831392]
 [-0.28465986]]


In [84]:
y[-4]*48+48

array([ 66.3444519 ,  38.45192337,  31.53630257,  37.64226532,
        59.59909439,  40.61041641,  72.28075409,  39.53086853,
        37.47260284,  39.80075455,  23.98128319,  38.99109268,
        53.67486954,  32.28679276,  81.8789444 ,  33.87230301,
        40.07064056,  31.16377449,  15.61660385,  33.05479431,
        48.26324463,  60.72513199,  63.10641479,  81.08498383,
        31.80618858,  79.19638062,  47.456604  ,  77.30716705,
        47.72649002,  85.40196228], dtype=float32)

In [117]:
# Generate df of resutls from each independent specialist net

pred = []
counter = 1
for net in allNets:
    #print '\r', 'Net: ', counter
    counter += 1
    pred.append(net.thisNet.predict(Xtest))

pred_df = pd.DataFrame(np.transpose(np.array(pred)[:,:,0])*48+48, columns = colnames)

In [142]:
#for row in range(len(pred_df)):
#    print pred_df.loc[row]
inds = pd.isnull(pred_df).any(1).nonzero()[0]
print inds

[]


In [143]:
outputFile = '../Data/dreamcoatSubmission5.csv'
oFile = open(outputFile,'w')
subform = open('../Data/submissionFormat.csv')
subLines = subform.readlines()
colnames = [u'left_eye_center_x', u'left_eye_center_y', u'right_eye_center_x',
       u'right_eye_center_y', u'left_eye_inner_corner_x',
       u'left_eye_inner_corner_y', u'left_eye_outer_corner_x',
       u'left_eye_outer_corner_y', u'right_eye_inner_corner_x',
       u'right_eye_inner_corner_y', u'right_eye_outer_corner_x',
       u'right_eye_outer_corner_y', u'left_eyebrow_inner_end_x',
       u'left_eyebrow_inner_end_y', u'left_eyebrow_outer_end_x',
       u'left_eyebrow_outer_end_y', u'right_eyebrow_inner_end_x',
       u'right_eyebrow_inner_end_y', u'right_eyebrow_outer_end_x',
       u'right_eyebrow_outer_end_y', u'nose_tip_x', u'nose_tip_y',
       u'mouth_left_corner_x', u'mouth_left_corner_y', u'mouth_right_corner_x',
       u'mouth_right_corner_y', u'mouth_center_top_lip_x',
       u'mouth_center_top_lip_y', u'mouth_center_bottom_lip_x',
       u'mouth_center_bottom_lip_y']

#pred_df = pd.DataFrame(yp2*48+48,columns=colnames)

header = subLines.pop(0)
header = 'rowId,Location'

lastLine = subLines.pop(-1)

oFile.write(header+'\n')
for ll in subLines:
    thisLine = ll.split(',')
    thisImageNum = int(thisLine[1])-1  # data are 1-indexed
    thisLabel = thisLine[2].strip()

    outLine = []
    outLine.append(str(thisLine[0]))
    thisLoc = pred_df[thisLabel][thisImageNum]
    
    if thisLoc > 96:
        print outLine,thisLoc
        thisLoc=96
        
    if thisLoc < 0:
        print outLine,thisLoc
        thisLoc = 0
      
    outLine.append(str('%.10f'%thisLoc))
    oFile.write(",".join(outLine)+'\n')
    #print ",".join(outLine)

oFile.close()
subform.close()
print 'Submission Generated.  File: ', outputFile

Submission Generated.  File:  ../Data/dreamcoatSubmission5.csv


In [ ]:
darf=SpecialistNet([colnames[-5]])

mouth_right_corner_y    2270
Image                   7049
dtype: int64
# Neural Network with 1658333 learnable parameters

## Layer information

  #  name     size
---  -------  --------
  0  input    1x96x96
  1  conv1    16x93x93
  2  pool1    16x46x46
  3  conv2    16x44x44
  4  pool2    16x22x22
  5  conv3    32x21x21
  6  pool3    32x10x10
  7  hidden4  500
  8  hidden5  100
  9  hidden6  30
 10  output   1

  epoch    train loss    valid loss    train/val  dur
-------  ------------  ------------  -----------  -----
      1       0.04575       0.00942      4.85762  2.45s
      2       0.00850       0.00942      0.90170  2.43s
      3       0.00844       0.00938      0.89975  2.44s
      4       0.00837       0.00927      0.90344  2.44s
      5       0.00827       0.00917      0.90132  2.44s
      6       0.00813       0.00905      0.89865  2.43s
      7       0.00796       0.00884      0.90068  2.43s
      8       0.00773       0.00854      0.90468  2.43s
      9       0.00742    

In [145]:
[colnames[-5]]

[u'mouth_right_corner_y']